# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 1 from hw3-t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [3]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [4]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [5]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [10]:

def flow_with_demands(graph):
    demands = []
    for state in graph.nodes():
        demands.append(graph.node[state]['demand'])

    if sum(demands) != 0:
        raise nx.NetworkXUnfeasible("No feasible Flow.")
        
    def initialize_the_flow(G):        # initialize the flow
        keys = []
        values = []
        for s1 in G.nodes():
            keys.append(s1)
            res = dict()
            for (s0, s2) in G.edges():
                if s0 == s1:
                    res[s2] = 0
            values.append(res)
        flow = dict()
        for i, k in enumerate(keys):
            flow[k] = values[i]
        return flow

    def Get_residual_graph(G, flow):        # Get the residual_graph
        G_res = nx.DiGraph()
        for (s1, s2) in G.edges():
            if flow[s1][s2] > 0: 
                G_res.add_edge(s2, s1)
                G_res[s2][s1]['capacity'] = flow[s1][s2]

            if flow[s1][s2] < G[s1][s2]['capacity']:
                G_res.add_edge(s1, s2)
                G_res[s1][s2]['capacity'] = G[s1][s2]['capacity'] - flow[s1][s2]

        return G_res

    def BFS(G, root):     # use BFS to find a simple_path in residual graph
        queue = [root]
        discovered = [root]
        prev = dict()
        while len(queue) > 0:
            u = queue[0]
            queue = queue[1:]
            for v in G.neighbors(u):
                if v not in discovered:
                    prev[v] = u
                    discovered.append(v)
                    queue.append(v)
        return [prev, discovered]

    def Find_Path(s, t, G):   # find the simple path using BFS above
        prev, discovered = BFS(G, s)
        if t not in discovered:
            return None
        else:
            path = [t]
            key = t
            while key != None:
                if key in prev.keys():
                    path.append(prev[key])
                    key = prev[key]
                else:
                    key = None
            return path[::-1]

    def Augment_flow(path, G_res, G, flow):  #update the flow dict according to the G_res calculatd above
        capacities = []
        for k in range(len(path) - 1):
            s1 = path[k]
            s2 = path[k + 1]
            capacities.append(G_res.edge[s1][s2]['capacity'])
        add_flow = min(capacities)
        for k in range(len(path) - 1):
            s1 = path[k]
            s2 = path[k + 1]
            if (s1, s2) in G.edges():
                flow[s1][s2] = flow[s1][s2] + add_flow
            else:
                flow[s2][s1] = flow[s2][s1] - add_flow
        return flow

    # create sourse 'S'
    G = graph.copy()
    G.add_node('S')
    for state in G.nodes():
        if state != 'S':
            if G.node[state]['demand'] < 0:
                G.add_edge('S', state)
                G.edge['S'][state]['capacity'] = -G.node[state]['demand']

    # create sink 'T'
    G.add_node('T')
    for state in G.nodes():
        if state != 'S' and state != 'T':
            if G.node[state]['demand'] > 0:
                G.add_edge(state, 'T')
                G.edge[state]['T']['capacity'] = G.node[state]['demand']


    flow = initialize_the_flow(G)
    G_res = Get_residual_graph(G, flow)
    path = Find_Path('S', 'T', G_res)
    while path != None:
        flow = Augment_flow(path, G_res, G, flow)
        G_res = Get_residual_graph(G, flow)
        path = Find_Path('S', 'T', G_res)

  
    demand = 0
    for state in G.nodes():
        if state != 'S' and state != 'T':
            if G.node[state]['demand'] > 0:
                demand += G.node[state]['demand']

    max_flow = 0
    for (s1, s2) in G.edges():
        if s1 == 'S':
            max_flow += flow['S'][s2]

    if max_flow == demand: # Find the flow we want & remove 'S' and 'T'
        for key in flow.keys():
            if 'T' in flow[key].keys():
                del flow[key]['T']
            if 'S' in flow[key].keys():
                del flow[key]['S']
        del flow['S']
        del flow['T']
        return flow
    else:
        raise nx.NetworkXUnfeasible("No Feasible Flow.")


{'WA': {'OR': 0, 'ID': 0}, 'DE': {'MD': 0, 'NJ': 0, 'PA': 0}, 'DC': {'MD': 0, 'VA': 0}, 'WI': {'IA': 0, 'MN': 0, 'MI': 1, 'IL': 0}, 'WV': {'MD': 0, 'OH': 0, 'VA': 0, 'PA': 0, 'KY': 0}, 'FL': {'AL': 0, 'GA': 0}, 'WY': {'CO': 0, 'UT': 0, 'NE': 16, 'MT': 0, 'ID': 0, 'SD': 2}, 'NH': {'ME': 1, 'MA': 0, 'VT': 0}, 'NJ': {'NY': 0, 'DE': 0, 'PA': 0}, 'NM': {'CO': 0, 'AZ': 0, 'OK': 4, 'TX': 2}, 'TX': {'LA': 1, 'AR': 0, 'OK': 0, 'NM': 0}, 'LA': {'AR': 0, 'MS': 0, 'TX': 0}, 'NC': {'SC': 1, 'TN': 0, 'GA': 0, 'VA': 0}, 'ND': {'MT': 0, 'MN': 2, 'SD': 0}, 'NE': {'CO': 0, 'MO': 14, 'KS': 0, 'IA': 1, 'WY': 0, 'SD': 0}, 'TN': {'VA': 4, 'MS': 0, 'MO': 0, 'NC': 2, 'AL': 2, 'AR': 0, 'GA': 1, 'KY': 0}, 'NY': {'NJ': 0, 'MA': 2, 'PA': 0, 'VT': 3, 'CT': 1}, 'PA': {'MD': 0, 'NJ': 1, 'OH': 0, 'DE': 0, 'WV': 0, 'NY': 7}, 'RI': {'MA': 0, 'CT': 0}, 'NV': {'UT': 15, 'CA': 0, 'ID': 0, 'AZ': 0, 'OR': 0}, 'VA': {'MD': 2, 'NC': 0, 'DC': 1, 'TN': 0, 'WV': 0, 'KY': 0}, 'CO': {'OK': 8, 'NM': 0, 'UT': 0, 'NE': 0, 'KS': 1, 'W

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [12]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    into_node = {}
    for k1 in flow.keys():
        if k1 not in into_node.keys():
            into_node[k1] = 0
        for k2 in flow[k1].keys():
            if k2 not in into_node.keys():
                into_node[k2] = 0
            into_node[k1] -= flow[k1][k2]
            into_node[k2] += flow[k1][k2]
    return into_node
    # TODO: Implement the function.
 

{'WA': 1, 'DE': 1, 'DC': 1, 'WI': 1, 'WV': 1, 'FL': 1, 'WY': 1, 'NH': 1, 'NJ': 1, 'NM': 1, 'TX': 1, 'LA': 1, 'NC': 1, 'ND': 1, 'NE': 1, 'TN': 1, 'NY': 1, 'PA': 1, 'RI': 1, 'NV': 1, 'VA': 1, 'CO': 1, 'CA': -48, 'AL': 1, 'AR': 1, 'VT': 1, 'IL': 1, 'GA': 1, 'IN': 1, 'IA': 1, 'OK': 1, 'AZ': 1, 'ID': 1, 'CT': 1, 'ME': 1, 'MD': 1, 'MA': 1, 'OH': 1, 'UT': 1, 'MO': 1, 'MN': 1, 'MI': 1, 'KS': 1, 'MT': 1, 'MS': 1, 'SC': 1, 'KY': 1, 'OR': 1, 'SD': 1}


The following code performs a sanity check on your function (but does not completely confirm correctness).

In [13]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
